In [ ]:
#The output of this notebook is a people df, where we have info about everyone in the simulaiton

#input : 
#> countries , a df with information about every country, you can freely edit it if you 
#> companies, a list of all companies
#> names
#> hobbies
#> banks
#> bank_accounts_ids : a list of every bank account with its related idn

In [4]:
import pandas as pd
import numpy as np
import string
import random 
import requests
from datetime import datetime
import json
import ast
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
 
countries = pd.read_excel('countries.xlsx')
 

names = pd.read_excel( 'names.xlsx')
hobbies = pd.read_excel( 'hobbies.xlsx')
# spending_beh_df= pd.read_excel( 'spendingBehaviour.xlsx')  

companies= pd.read_excel( 'companies.xlsx')  

 
list_of_countries= list( companies.country.unique())


banks=  pd.read_excel( 'banks.xlsx' )  


bank_accounts_ids  = pd.read_excel( 'bank_accounts_ids.xlsx' )   
transactions=  pd.read_excel( 'transactions.xlsx' )  

ai_traininig_df = pd.read_excel('training_data/ai_traininig_df_final.xlsx')


print ('done')

 
 

done


In [7]:
#functions:


#return a unique idn, you can mention where to search 
#you can decide the nr_digits and nr_letters in the idn
def unique_idn(nr_digits=10, nr_letters=0, search_list=[]):
#         idn= "".join(list (map(str, list( np.random.randint(0,10,nr_digits)))))
        letters =list ( np.random.choice(list (string.ascii_letters), size=nr_letters, replace=True))
        digits = list (np.random.choice(list( string.digits), size=nr_digits  ))
        idn=letters + digits


        random.shuffle(idn)
        idn="".join (idn) 
        
    #if exists: return the function again
        if idn in search_list :
           #print('not unique')
           return unique_idn(nr_digits,search_list)
        return idn
unique_idn(nr_digits=4,nr_letters=3)




'v5413Nh'

In [8]:
#returns a random name from names df, takes the country into acc
def get_random_name(country, gender):
    # Find the row corresponding to the country
    country_row = names[names['country'] == country]
        
    if gender == 'male':
        name_list = ast.literal_eval(country_row['male_names'].values[0]) 
    elif gender == 'female':
        name_list =  ast.literal_eval(country_row['female_names'].values[0]) 
     
    
    # Return a random name from the list
    return random.choice(name_list)

In [9]:
#takes a string, return a list of corresponding numbers, every charachter is mapped to a number. 
#This is necessary to conclude the check_digits for every iban
def convert_to_numbers( strng):
    strng=strng.lower()
    # a dict that maps each letter to a number
    letter_to_number = {letter: index + 10 for index, letter in enumerate(string.ascii_lowercase)}
    list_numbers=[]
    for c in strng:
        if c.isdigit():
            list_numbers.append(c)
        else:
            list_numbers.append(str (letter_to_number[c]))
    return list_numbers
    
 
convert_to_numbers('60161331926819GB29')



['6',
 '0',
 '1',
 '6',
 '1',
 '3',
 '3',
 '1',
 '9',
 '2',
 '6',
 '8',
 '1',
 '9',
 '16',
 '11',
 '2',
 '9']

In [10]:





# return an iban ( also acc_nr) 



def generate_new_bank_acc(country_code='NL',bank_code='', location_code='' , acc_nr=''): 
          #genertate an account number: random 12 digits 
        acc_nr= "".join(list (map(str, list( np.random.randint(0,12,10)))))
         
         

        
            #the iban is CCkk bbbb ssss cccc cccc cccc where:
        #     CC: Country Code (2 letters)
        # kk: Check Digits (2 digits)
        # bbbb: Bank Code (varies by country)  
        # ssss: Branch Code (optional, varies by country)
        # cccc cccc cccc: Account Number (the individual bank account number)
        
        #for the bbbb, we will get it from the banks dataset, or you can generate a random number of 4 digits


        #first form the strng of the IBAN : <<to do >> check the form of the list
        strng = f'{location_code}{acc_nr}{bank_code}{country_code}'
        #then calculate the check code : 
          #first convert to one big number
        big_nr= int("".join (convert_to_numbers(strng))  )  
          #Perform Modulo 97 Calculation : 98 - ( remainder when deviding by 97)
        remainder97= big_nr%97
        check_digits =98- remainder97
        iban = f'{country_code}{check_digits}{bank_code}{location_code}{acc_nr}'
        currency = countries.query("country_code==@country_code").currency.values[0]

     #  <<TODO:  make sure it is unique >>: search for that iban in the bank_accounts_ids df, 
    #if exists: return the function again
        if iban in bank_accounts_ids.iban.tolist():
          # print('not unique')
           return generate_new_bank_acc(country_code=country_code,bank_idf='', location_code='' , acc_nr='')

       # some ibans have less charachtaristics than others!
   # <<TO DO: fix this problem>>
        
    ##  
        return {'iban':iban, 
               'acc_nr':acc_nr,
               'country_code':country_code,
               'currency':currency,
               }  
    
# person= generate_person()
generate_new_bank_acc()    
# generate_iban(country_code,bank_idf, location_code , acc_nr)

{'iban': 'NL2011089764524',
 'acc_nr': '11089764524',
 'country_code': 'NL',
 'currency': 'EUR'}

In [53]:
#will generate a first_name last_name, 
def generate_person():
        global bank_accounts_ids
    #first a unique idn: 
        idn= unique_idn(nr_digits=10, search_list=bank_accounts_ids.idn.tolist()) 
       

       
        #select a random country based on the birth rate
        country= countries.sample(n=1, weights='Birth_Rate')['country'].iloc[0]
        #country_code
        country_code = countries.query("country==@country")['country_code'].iloc[0]
        exchange_rate = countries.query("country==@country")['exchange_rate'].values[0]
        
        
       
       
         
        
        #decide gender, based on mf_ratio: <<TO DO : add other gender>>
        male_prob = countries.query("country == @country")['male_percentage'].values[0]
        gender = np.random.binomial(n=1, p=male_prob)
        
        
            
        #generate age based on average age in that counrty: first get the mean age for this country
        #also convert it to male and female
        if gender==0:
            mean_age= countries.query( "country== @country")['mean_age_m'].values[0]
            gender='male'
        else:
             mean_age= countries.query( "country== @country")['mean_age_f'].values[0]
             gender='female'
        
        #name
        first_name=   get_random_name(country , gender )
        last_name =get_random_name(country , 'male' )
        
        
        # sample from a normal distribution with mean = mean_age  
        age = int (round (float( np.random.normal(loc=mean_age, scale=mean_age*0.2, size= 1)),2))
        if age < 20 : age =20
        
        # now education level :  #m1:    age, gender, country > education level
        #$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$ m1
        try:
            data_m1 = {
            "age": age,
            "gender":  gender,
            "country":  country 

            }
            response = requests.post("http://localhost:8000/predict_m1", data=data_m1)
            m1=response.text
            education_level  =  json.loads(m1)[0]
        except:
             education_level = 'Postgraduate'
        if education_level ==None or education_level in [  'Vocational', 'PhD'] : 
            education_level = 'Postgraduate'
           
        #$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$ m1
        
#         print(f'education is {education}')


         



        #choose a specific title : Master of Data Science for example
        #here: this is m2: just a random choice from ai_training_df with some assumptions:
        
        ####################%%%%%%%%%%%%%%%%%%%%%%%%%% m2
        list_edu_titles= ai_traininig_df.query("education_level==@education_level and country ==@country and age<=@age").education_title.tolist()
         
        #relaxing some of the conditions: i.e. removing the country condition
        if len(list_edu_titles) ==0 :
            return generate_person() 
            
        education_title= np.random.choice( list_edu_titles)
        education_title
        ####################%%%%%%%%%%%%%%%%%%%%%%%%%% m2
        
        #&&&&&&&&&&&&&&&&&&&&&&&&&&& m3
        #now based on education : a profession is decided ( also a random choice with some assumptions ) : 
        list_professions = ai_traininig_df.query("education_title==@education_title ").profession.tolist()
        profession=np.random.choice ( list_professions )
        #&&&&&&&&&&&&&&&&&&&&&&&&&&& m3
        
        
        #now  decide "previous" years of experience
        #@@@@@@@@@@@@@@@@@ m4 
        try:

            data_m4 = {
                "age": age,
                "gender": gender,
                "country": country,
             "profession" :  profession
            }
            response = requests.post("http://localhost:8000/predict_m4", data=data_m4)
            m4=response.text

            years_experience = int ( json.loads(m4) ['predictions'][0])
        except:
            years_experience =1
        #@@@@@@@@@@@@@@@@@ m4 
        if age-18 < years_experience: 
            years_experience = age-18  # assuming he started working at 18
        
        
         
        
        #now working (currently): yes or no ?  based on all those factors 
        #^^^^^^^ m5
        try:  
            data_m5 = {
            "age": age,
            "gender": gender,
            "country":  country,
            "profession" :  profession, 
            "years_experience" : years_experience

            }
            response = requests.post("http://localhost:8000/predict_m5", data=data_m5)
            m5=response.text
            working = json.loads(m5) ['predictions'][0]
        except:
            working ='1'
        #^^^^^^^ m5
        
        
        
        #print(f'working is {working}')
        
        #%%%%%%%%%%%%%^^^^^^^^^^^^^ m6
        #m6
        try:
            data_m6 = {

            "gender": gender,
            "country": country,
            "profession" :  profession, 
            "years_experience" : years_experience

            }
            response = requests.post("http://localhost:8000/predict_m6", data=data_m6)
            m6=response.text
            salary_eur = int  (json.loads(m6)[0])
            salary = exchange_rate * salary_eur
        except:
            salary_eur=1000
            salary = exchange_rate * salary_eur
        #%%%%%%%%%%%%%^^^^^^^^^^^^^ m6
        
        
#         working = 0 #to be removed later
       # couple with a working company and decide   source_of_income : we do that here to adjust the salary if needed
        if working==0:
            id_work_company= np.random.choice (companies.query("country==@ country and high_level_category=='Government'")['idn'].tolist())
            source_income ='social_benefits' 
            minimum_survival=  countries.query("country==@country")['minimum_survival'].values[0]
 
            salary = 1.2 * minimum_survival
            salary_eur = salary/ exchange_rate
        else:    
            source_income=  'work'
            id_work_company= np.random.choice (companies.query("country==@ country")['idn'].tolist())
        work_company= companies.query("idn==@id_work_company").company.values[0]
         
        
        
        #!!!!!!!!!!!!!!!!!!!!!!! m7
        #m7
        # balance ~ salary_eur,working,education_level, gender , country, profession, years_experience
        try:
            data_m7 = {
            'salary_eur' : salary_eur,
            'working' :  working,
            'education_level' : education_level,
            "gender":  gender,
            "country":  country,
            "profession" :  profession, 
            "years_experience" : years_experience

            }
            response = requests.post("http://localhost:8000/predict_m7", data=data_m7)
            m7=response.text
            balance_eur = int  (json.loads(m7)[0])
            balance = balance_eur * exchange_rate  
        except:
            balance =10000
            balance_eur =exchange_rate * balance
        #!!!!!!!!!!!!!!!!!!!!!!! m7
        
        #spending_behaviour  m8 : 
        
        
        #&**************** m8
        
            #m8
        #   spendingBehaviour ~ balance_eur +   working + education_level+   gender + country + profession,
        try:
            data_m8 = {
            'balance_eur' : balance_eur,
            'working' : working,
            'education_level' : education_level,
            "gender":  gender ,
            "country": country,
            "profession" :  profession, 


            }
            response = requests.post("http://localhost:8000/predict_m8", data=data_m8)
            m8=response.text
            spendingBehaviour=  json.loads(m8) ['predictions'][0]
        except:
            spendingBehaviour= 'Middle-Class_Value_Seeker'

        #&**************** m8
        
        
         
            
        
        
        
      #  <<<<<<<<<TO DO>>>>>>>>>>>>
    #hobbies
        hobby = np.random.choice( hobbies['hobby'].tolist())
  
    
            
            
        
         
#couple with necessity companyies : housing, health ,energy, internet
        id_housing_company=np.random.choice (companies.query("country==@country and high_level_category=='Housing'").idn.tolist())
        id_energy_company= np.random.choice  (companies.query("country==@country and high_level_category=='Energy'").idn.tolist())
        id_health_ins_company =  np.random.choice  (companies.query("country==@country and high_level_category=='Health Insurance'").idn.tolist())
        id_internet_company= np.random.choice  (companies.query("country==@country and high_level_category=='Internet'").idn.tolist())
     
    
        housing_company =      companies.query("idn==@id_housing_company").company.values[0]
        
        
        energy_company=      companies.query("idn==@id_energy_company").company.values[0]
        health_ins_company=   companies.query("idn==@id_health_ins_company").company.values[0]
        internet_company=    companies.query("idn==@id_internet_company").company.values[0]
       #this should be changed : select randomly the id and then get the name! to avoid too many conditions
    
       
#decide amounts for all those
#first get mean_costs and then sample from a normal distribution centered around the mean
# <<TO DO : costs should also be predicted, based on salary, balance, mean costs ... etc >>

	 		 	 

        mean_monthly_rent_costs= countries.query("country==@country")['mean_monthly_rent_costs'].values[0]
        mean_monthly_energy_costs= countries.query("country==@country")['mean_monthly_energy_costs'].values[0]
        mean_monthly_health_ins_costs= countries.query("country==@country")['mean_monthly_health_ins_costs'].values[0]
        mean_monthly_internet_costs= countries.query("country==@country")['mean_monthly_internet_costs'].values[0]
        mean_monthly_groceries_costs= countries.query("country==@country")['mean_monthly_food_costs'].values[0]

        monthly_rent_costs=round (float( np.random.normal(loc=mean_monthly_rent_costs, scale= mean_monthly_rent_costs*0.2 , size= 1)),2)
        monthly_energy_costs= round (float( np.random.normal(loc=mean_monthly_energy_costs, scale= mean_monthly_energy_costs*0.2, size= 1)),2)
        monthly_health_ins_costs =round (float( np.random.normal(loc=mean_monthly_health_ins_costs, scale= mean_monthly_health_ins_costs*0.1, size= 1)),2)
        monthly_internet_costs = round (float( np.random.normal(loc=mean_monthly_internet_costs, scale= mean_monthly_internet_costs*0.1, size= 1)),2)
        monthly_groceries_costs = round (float( np.random.normal(loc=mean_monthly_groceries_costs, scale= mean_monthly_groceries_costs*0.2, size= 1)),2)
        
       
 

 
    
    
    
    
    #couple with a bank:
   # <<TO DO>>
        bank_company=  banks.query("country==@country").sample(n=1)  
        bank_name = bank_company.company.values[0] 
        bank_code =bank_company.bank_code.values[0]   # bank['bank_code']
        bank_idn = bank_company.idn.values[0] 
    
    #make a bank account : 
        bank_acc= generate_new_bank_acc(country_code=country_code,bank_code=bank_code, location_code='' , acc_nr='')
        #now after we have the bank_acc 
        bank_acc['balance']= balance
        
        #now export the bank acc details to the bank_accounts_ids df:
        
        bank_accounts_ids_new_dict = {'idn':idn,
#   this is not neccessary(you will have to update it later, so skip for now)   	'bank_acc':	bank_acc,
                                      'iban': bank_acc['iban'],
                                      'acc_nr':bank_acc['acc_nr']  ,
                                      'currency': bank_acc['currency'],
                                      'country_code': country_code,
                                      'country':country,
                                      'balance': balance ,
                                     'bank_name': bank_name,
                                     'bank_idn':bank_idn }
  
#         print ( bank_accounts_ids_new_dict)      
        bank_accounts_ids_new  = pd.DataFrame([bank_accounts_ids_new_dict])
#         print ( bank_accounts_ids_new ) 
       
        bank_accounts_ids  = pd.concat([bank_accounts_ids,bank_accounts_ids_new  ], ignore_index=True)        
      
 
        
    
# <<to do > : move anything related to the country ( like the mean costs) to a df of country profile

        
        info=  {'idn':idn,
                'first_name':first_name,
                'last_name':last_name,
                'gender':gender,  
                'country':country, 
                'country_code':country_code,
                 'exchange_rate':exchange_rate,
                'male_prob':male_prob, 
                'mean_age': mean_age, 
                'age':age,
                'education_level':education_level,
                'education_title':education_title,
                'profession':profession,
                'years_experience':years_experience,
                'working':working,
                'source_income':source_income,
                 'salary_eur': salary_eur,
                'salary':salary,
                 'initial_balance_eur' : balance_eur, #changed the name to initial because this will not be updated in transactions
                'initial_balance':balance,
                'hobby': hobby,
                'spendingBehaviour':spendingBehaviour,
                'work_company':work_company,
                'id_work_company':id_work_company,
                'id_housing_company':id_housing_company,
                'id_energy_company':id_energy_company,
                'id_health_ins_company':id_health_ins_company,
                'id_internet_company':id_internet_company,


                'housing_company':housing_company,
                'energy_company':energy_company,
                'health_ins_company':health_ins_company,
                'internet_company':internet_company,

                'mean_monthly_rent_costs':mean_monthly_rent_costs,
                'mean_monthly_energy_costs':mean_monthly_energy_costs,
                'mean_monthly_internet_costs' :mean_monthly_internet_costs,
                'mean_monthly_health_ins_costs':mean_monthly_health_ins_costs,
                'mean_monthly_groceries_costs':mean_monthly_groceries_costs,
                
                'monthly_rent_costs':monthly_rent_costs,
                'monthly_energy_costs':monthly_energy_costs,
                'monthly_internet_costs':monthly_internet_costs,
                'monthly_health_ins_costs':monthly_health_ins_costs,
                'monthly_groceries_costs' : monthly_groceries_costs ,
                'bank_name' : bank_name,
#                  'bank_company':bank_company, #to be removed
                 'bank_idn':bank_idn,
                #those will be moved to the accounts_id df
#                                  'iban':bank_acc['iban'],
#                                   'bank_acc_cur':bank_acc['currency'],
#                                  'bank_acc':bank_acc,
                 #####
                'id_housing_company':id_housing_company,
                'id_energy_company':id_energy_company,
                'id_health_ins_company':id_health_ins_company,
                'id_internet_company':id_internet_company,

                }
#         print(info)
        return info

generate_person()



{'idn': '2892698356',
 'first_name': 'Samuel',
 'last_name': 'Paul',
 'gender': 'male',
 'country': 'Malta',
 'country_code': 'MT',
 'exchange_rate': 1.0,
 'male_prob': 0.5,
 'mean_age': 41.7,
 'age': 42,
 'education_level': 'Postgraduate',
 'education_title': 'Master of International Relations',
 'profession': 'Diplomat',
 'years_experience': 1,
 'working': '1',
 'source_income': 'work',
 'salary_eur': 1000,
 'salary': 1000.0,
 'initial_balance_eur': 10000.0,
 'initial_balance': 10000,
 'hobby': 'Rock climbing',
 'spendingBehaviour': 'Middle-Class_Value_Seeker',
 'work_company': 'Piadina Café',
 'id_work_company': 4136857984,
 'id_housing_company': 2682096435,
 'id_energy_company': 7840755992,
 'id_health_ins_company': 3727638267,
 'id_internet_company': 4941057787,
 'housing_company': 'Tigné Point',
 'energy_company': 'Enemalta',
 'health_ins_company': 'Allianz Malta',
 'internet_company': 'Melita Communications',
 'mean_monthly_rent_costs': 900,
 'mean_monthly_energy_costs': 150,
 '

Generating People:
 

In [ ]:
columns =  generate_person().keys()
# Initialize an empty DataFrame with the specified columns
nr_people = 3000
people1 = pd.DataFrame(columns=columns)

for j in range(nr_people): 
     
     
     
        
    people1= pd.concat([people1, pd.DataFrame([generate_person()]) ], ignore_index=True)
        
people1.to_excel(   "people.xlsx",   index=False)
bank_accounts_ids.to_excel('bank_accounts_ids.xlsx', index=False)

        
print ( f'generated {nr_people}people ')  

In [1]:
#here if you want to generate in in parts:

# columns =  generate_person().keys()
# # Initialize an empty DataFrame with the specified columns
# for j in range(6): 
#     people1 = pd.DataFrame(columns=columns)
#     print ( f"&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&& j is {j}    &&&&&&&&&&&&&&&&&&&&&&")

#     for i in range (500):
#        print (i)
#        people1= pd.concat([people1, pd.DataFrame([generate_person()]) ], ignore_index=True)
        
#     people1.to_excel(  f"people_{j }.xlsx",   index=False)
#     bank_accounts_ids.to_excel('bank_accounts_ids.xlsx', index=False)
 
        
#     print ( f'people_{j} is done')  